In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
items_25 = pd.read_csv("../Final Project/items_2025.csv")
prices_25 = pd.read_csv("../Final Project/prices_2025_cross_section.csv")

In [ ]:
male_words = ['man', 'men', 'male', 'masculine', 'boy', 'boys', 'him', 'his', 'he']
female_words = ['woman', 'women', 'female', 'feminine', 'girl', 'girls', 'her', 'hers', 'she']

In [ ]:
items_25_mf = items_25

In [ ]:
def words_in_string(word_list, title):
    if str(set(word_list).intersection(title.split())) == 'set()':
        return False
    else:
        return True

In [ ]:
items_25_mf['m'] = 0
items_25_mf['f'] = 0
items_25_mf['n'] = 0
items_25_mf['b'] = 0

for i in range(len(items_25_mf['product_title'])):
    pt = items_25_mf['product_title'][i]
    pt_std =  re.sub(r'[^\w\s]','',pt.lower())
    if (words_in_string(male_words, pt_std) == True) & (words_in_string(female_words, pt_std) == False): # If there is a male word but not a female word
        items_25_mf['m'][i] = 1
    elif (words_in_string(male_words, pt_std) == False) & (words_in_string(female_words, pt_std) == True): # If there is a female word but not a male word
        items_25_mf['f'][i] = 1
    elif (words_in_string(male_words, pt_std) == False) & (words_in_string(female_words, pt_std) == False): # If there is neither a male or female word
        items_25_mf['n'][i] = 1
    elif (words_in_string(male_words, pt_std) == True) & (words_in_string(female_words, pt_std) == True): # If there is both a male and female word
        items_25_mf['b'][i] = 1
    else:
        print(f'{i}');

In [ ]:
print(f'male items are {round(items_25_mf['m'].sum()/len(items_25_mf) * 100,2)}% of cases')
print(f'female items are {round(items_25_mf['f'].sum()/len(items_25_mf) * 100,2)}% of cases')
print(f'neutral items are {round(items_25_mf['n'].sum()/len(items_25_mf) * 100,2)}% of cases')
print(f'both items are {round(items_25_mf['b'].sum()/len(items_25_mf) * 100,2)}% of cases')

In [ ]:
m_list = []
f_list = []
n_list = []
b_list = []

for i in range(len(items_25_mf['product_title'])):
    pt = items_25_mf['product_title'][i]
    pt_std =  re.sub(r'[^\w\s]','',pt.lower())
    if items_25_mf['m'][i] == 1:
        m_list.append(pt_std.split())
    if items_25_mf['f'][i] == 1:
        f_list.append(pt_std.split())
    if items_25_mf['n'][i] == 1:
        n_list.append(pt_std.split())
    if items_25_mf['b'][i] == 1:
        b_list.append(pt_std.split())

In [ ]:
m_list = [word for title in m_list for word in title]
f_list = [word for title in f_list for word in title]
n_list = [word for title in n_list for word in title]
b_list = [word for title in b_list for word in title]

In [ ]:
def word_freq(x_list, x_list_count):
    #x_list_count = []
    for word in set(x_list):
        x_list_count.append([word,x_list.count(word)/len(x_list)])

In [ ]:
m_list_count = []
f_list_count = []
n_list_count = []
b_list_count = []

In [ ]:
word_freq(m_list,m_list_count)
word_freq(f_list,f_list_count)
word_freq(n_list,n_list_count)
word_freq(b_list,b_list_count)

In [ ]:
m_df = pd.DataFrame(m_list_count, columns= ['word','m_titles_freq'])
f_df = pd.DataFrame(f_list_count, columns= ['word','f_titles_freq'])
n_df = pd.DataFrame(n_list_count, columns= ['word','n_titles_freq'])
b_df = pd.DataFrame(b_list_count, columns= ['word','b_titles_freq'])

In [ ]:
comb_df = m_df.merge(f_df, how='outer',on='word').merge(n_df, how='outer', on='word').merge(b_df, how='outer', on='word')

In [ ]:
comb_df['m_titles_freq'] = comb_df['m_titles_freq'].fillna(0)
comb_df['f_titles_freq'] = comb_df['f_titles_freq'].fillna(0)
comb_df['n_titles_freq'] = comb_df['n_titles_freq'].fillna(0)
comb_df['b_titles_freq'] = comb_df['b_titles_freq'].fillna(0)

In [ ]:
comb_df['m_freq_net'] = comb_df['m_titles_freq'] -  comb_df['n_titles_freq']
comb_df['f_freq_net'] = comb_df['f_titles_freq'] -  comb_df['n_titles_freq']
comb_df['b_freq_net'] = comb_df['b_titles_freq'] -  comb_df['n_titles_freq']

In [ ]:
comb_df_fil = comb_df[~comb_df['word'].isin(male_words)]
comb_df_fil = comb_df_fil[~comb_df_fil['word'].isin(female_words)]

In [ ]:
comb_df_fil.sort_values('m_freq_net', ascending=False)['word'][0:10]

In [ ]:
comb_df_fil.sort_values('f_freq_net', ascending=False)['word'][0:10]